### Read the Tx data file. 

Points to be noted:

- The transaction dataset contains all data merged from OMS and ECOM dataset across 2020,2021 and 2022

1. For 2021 First touch channel information is not available so appears as NaN, but this is anyways not important for our analysis

2. We will create a column called sequence which will assign a logical sequence to each month+year combination.

3. We will also create a channel called Channel type which will indicate if the channel is Paid/Organic channel type

In [41]:
df_oms_ecom=pd.read_csv('Transaction_Data.csv')

df_oms_ecom.loc[df_oms_ecom['Year']==2021,'Year']=0 #Year 2021, year value is 0
df_oms_ecom.loc[df_oms_ecom['Year']==2022,'Year']=12 #Year 2022, year value is 12
df_oms_ecom.loc[df_oms_ecom['Year']==2023,'Year']=24 #Year 2023, year value is 24
df_oms_ecom['Seq']=df_oms_ecom['Month']+df_oms_ecom['Year']
df_oms_ecom.head()

#Add a column called Channel type to understand whether the channel type was Paid/Organic channel
Paid=['Paid Search','TCP','Affiliate','Referral','Paid Social','CLM','Display','Privilege','Store']
Organic=['Organic Search','Direct','Internal','Organic Social',]

df_oms_ecom['Channel_Type']=df_oms_ecom['First Touch Channel'].apply(lambda x:'Paid' if x in Paid
                                                                     else 'Organic' if x in Organic
                                                                    else 'Unknown')

#Since 2021 Marketing channel is not important
df_oms_ecom[(df_oms_ecom['Seq']>12) & (df_oms_ecom['Seq']<29)].head()

<IPython.core.display.Javascript object>

,OrderNo,OrderDate,Month,Year,CustomerID,Revenue,Seq,First Touch Channel,Channel_Type
1,300031636886,2022-08-15,8,12,eb8912792390dd07164983b61e57f9f3,699.00,20,Paid Search,Paid
2,300031642826,2022-08-15,8,12,88f89b619ef1e08ce0936e1617c29def,683.14,20,Referral,Paid
3,300030856658,2022-08-15,8,12,b3be32338bac13df4b48ffef66685116,1350.00,20,Paid Search,Paid
4,300031675453,2022-08-15,8,12,cbfa015c137ae64c0da3b0712c3e4096,7499.00,20,Paid Search,Paid
5,300031673349,2022-08-15,8,12,d0e699dd67143350237608a6fc6ada07,18999.00,20,Paid Search,Paid


### Part 1 Find out  Newly acquired customers in Performace Window(January-May 2022)

Identify who are the new customers in Jan-May 2022. To do this 
1. we will look at the first and last purchase of each customer. 
2. We will find out the difference between the first and last purchase of each customer.
3. We will discard all customers whose difference in month between FP and LP is <=12. Exception is if Diff=0, since then it would indicate either a single purchase or a purchase made in the same month
4. From this set we will keep the data only for Jan-May 22

This way we get all customers who made their first purchase between Jan-May and did not make a purchase anytime in the last 12 months

In [7]:
#Create a dataframe from Jan 21 to May22
df_jan21_may22=df_oms_ecom[(df_oms_ecom['Seq']>=1) & (df_oms_ecom['Seq']<=17)]
#Sort the dataframe by customer, order date, order id so that the dataframe is sorted by Orderdate sequence of each customer
df_jan21_may22.sort_values(by=['CustomerID','OrderNo','OrderDate'],ascending=True,inplace=True)

#Find out each customers first and last purchase in this window
df_first=df_jan21_may22.groupby(by='CustomerID').first().reset_index()
df_first=df_first[['CustomerID','Seq']]
df_first.rename(columns={'Seq':'First'},inplace=True)

#Find out the difference in month between the 1st and last purchase of each customer
df_last=df_jan21_may22.groupby(by='CustomerID').last().reset_index()
df_last=df_last[['CustomerID','Seq']]
df_last.rename(columns={'Seq':'Last'},inplace=True)

#Merge the 2  dataframes to get the Seq of the first and last purchase made by the customer
df_diff=df_first.merge(df_last,on='CustomerID')

#If difference is 0 retain, if difference is >12 retain else drop
df_diff['Diff']=df_diff['Last']-df_diff['First']
df_diff=df_diff[(df_diff['Diff']==0) | (df_diff['Diff']>12)]

#Retain those customer ids whose 1st purchase is between Jan 22 to May 22.
df_new_jan_may_2022=df_diff[(df_diff['First']>12) & (df_diff['First']<=17)]


#How many customers are newly acquired in Performance window and what % of total population is that?
df_jan_may_2022=df_oms_ecom[(df_oms_ecom['Seq']>12) & (df_oms_ecom['Seq']<=17)]
a=df_jan_may_2022['CustomerID'].nunique()
#Total customers
b=df_new_jan_may_2022['CustomerID'].nunique()
print('New customers in Performance window:',b)
print('%New customers of all customers in Performance Window:',b/a)

#Find out transactions made by New customers across the Performance window
df_jan22_may22=df_oms_ecom[(df_oms_ecom['Seq']>12) & (df_oms_ecom['Seq']<=17)]
df_jan22_may22=df_new_jan_may_2022.merge(df_jan22_may22,on='CustomerID')#retain tx only for New customers as in P1
print('Transactions made by New customers in the performance window:',df_jan22_may22['OrderNo'].nunique())

C:\Users\24057\AppData\Local\Temp\ipykernel_20628\320892060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan21_may22.sort_values(by=['CustomerID','OrderNo','OrderDate'],ascending=True,inplace=True)


New customers in Performance window: 220694
%New customers of all customers in Performance Window: 0.8431158312958436
Transactions made by New customers in the performance window: 254368


### Part 2: Filter the data to retain customers whose 1st transaction was Paid

Since we will be calculating the ROI of a customer acquired by a PAID channel we will need to ensure the first transaction made in the performance window is Paid channel driven

In [8]:
#Find out the first transaction made by each customer in Jan-May
df_jan22_may22.sort_values(by=['CustomerID','OrderDate'],ascending=True,inplace=True)
df_cust_first=df_jan22_may22.groupby(by='CustomerID').first().reset_index()

#Retain only those transactions where the channel type is Paid
df_cust_first_paid_tx=df_cust_first[df_cust_first['Channel_Type']=='Paid']

df_cust_first_paid_tx=df_cust_first_paid_tx[['CustomerID','OrderNo','OrderDate','Month','Year','First Touch Channel',
                                      'Channel_Type']]

print("New customers who made a Paid transaction in the performance window:",df_cust_first_paid_tx['CustomerID'].nunique())
print("% of new customers who made a Paid transaction:",
      df_cust_first_paid_tx['CustomerID'].nunique()/df_cust_first['CustomerID'].nunique())

df_cust_first_paid_tx.head(10)

New customers who made a Paid transaction in the performance window: 100049
% of new customers who made a Paid transaction: 0.45333810615603504


,CustomerID,OrderNo,OrderDate,Month,Year,First Touch Channel,Channel_Type
0,00001d84fb7190e6300d5a77984e6f3c,300025829739,2022-05-04,5,12,Paid Search,Paid
1,0000798fe5f3201cadf596363e660435,300022118848,2022-01-25,1,12,Affiliate,Paid
4,0000cf33c48c5d7973cdef3c474ed871,300023637078,2022-03-17,3,12,Paid Search,Paid
5,000141c7d0784e5c27f5bacb16e4821e,300025869289,2022-05-06,5,12,Affiliate,Paid
12,0002b23932a37e0ceab9099cd0e55696,300022075838,2022-01-24,1,12,Paid Search,Paid
13,0002b8a442c82d47e68b1a3552388f15,300025505098,2022-04-28,4,12,Paid Search,Paid
16,0002f570873115f4751d8db02811bcd1,300022523452,2022-02-09,2,12,Paid Search,Paid
21,00048f028e9c82b572563626d98d08aa,TC300026065406,2022-05-09,5,12,TCP,Paid
22,00049ab42e333dd915497be2d5937cde,TC300026818448,2022-05-21,5,12,TCP,Paid
23,0004dadde0b566e79cfe1b26e1f603c4,300024048804,2022-03-31,3,12,Paid Search,Paid


### Part 3: Find out all transactions made by the filtered customers across the lifetime

In [9]:
#We will retain the Customer ID information from the df_cust_first_paid_tx table
df_cust_first_paid_tx=df_cust_first_paid_tx[['CustomerID','OrderNo']]
df_cust_first_paid_tx.rename(columns={'OrderNo':'FirstOrderNo'},inplace=True)
#Create the lookforward window such that all transactions in the lifetime are available
df_jan22_may23=df_oms_ecom[(df_oms_ecom['Seq']>12) & (df_oms_ecom['Seq']<=29)]


#Retain data for the filtered Customer IDs in Lifetime window
df_jan22_may23=df_jan22_may23.merge(df_cust_first_paid_tx,on='CustomerID')

#Additionally we will add a column called Flag to the frame so that we know which row is the first order
df_jan22_may23['Flag']=df_jan22_may23['OrderNo'] ==df_jan22_may23['FirstOrderNo']

df_jan22_may23['Flag']=df_jan22_may23['Flag'].apply(lambda x:0 if x==False
                             else 1 if x==True
                             else 'Unknown')

print('Transactions made by New customers across lifetime:',df_jan22_may23['OrderNo'].nunique())
df_jan22_may23.sample(5)

Transactions made by New customers across lifetime: 155343


,OrderNo,OrderDate,Month,Year,CustomerID,Revenue,Seq,First Touch Channel,Channel_Type,FirstOrderNo,Flag
123680,300021538961,2022-01-02,1,12,39f6b44e1cbb216cfaeb8f70067158ef,1500.05,13,Paid Search,Paid,300021538961,1
62879,300030535098,2022-07-30,7,12,53cc5dabb8b5acc9fa7dee4e8c3f76ba,21790.00,19,NaN,Unknown,300025523065,0
47784,TC300028774860,2022-06-27,6,12,9d9a69f05dcef2178c922a3affe1b332,9024.05,18,Affiliate,Paid,TC300027228741,0
106311,300043564397,2023-02-26,2,24,547605a20c1703ecde76cb4814a03a6a,999.00,26,NaN,Unknown,300024225643,0
21718,300025661366,2022-05-01,5,12,a68afcd834f1964a58aa56be92f0aec9,1149.00,17,Paid Search,Paid,300025661366,1


### Part 4: Of all the purchases made by the New customers retain only Organic repeat purchases and Paid first time purchases

Now that we have the Customer IDs who qualify for a Paid purchase in the window and have the flags set for new and return purchase we will retain:

1. If flag=1 then Channel type should be Paid
2. If flag =0 then Channel type should be Organic


In [11]:
df_jan22_may23=df_jan22_may23[(df_jan22_may23['Flag']==1) 
                        | ((df_jan22_may23['Flag']==0)
                           & (df_jan22_may23['Channel_Type']=='Organic'))]


print("#Total transactions made by new customers when Repeat paid transactions removed",
      df_jan22_may23['OrderNo'].nunique())

df_jan22_may23.head()

#Total transactions made by new customers when Repeat paid transactions removed 110672


,OrderNo,OrderDate,Month,Year,CustomerID,Revenue,Seq,First Touch Channel,Channel_Type,FirstOrderNo,Flag
1,300026821121,2022-05-21,5,12,669a59c305f29fdd332fcf85d2b41d66,5509.80,17,Paid Search,Paid,300026821121,1
7,TC300025956228,2022-05-07,5,12,133afa43f4aca20d1541b47bbbceea59,44990.00,17,TCP,Paid,TC300025956228,1
12,TC300026763825,2022-05-20,5,12,ba789948cd094725bde4064809c8d25c,54990.00,17,TCP,Paid,TC300026763825,1
17,300024588642,2022-04-07,4,12,a0461e874d88d8ff4df469ba5bb1e108,3134.05,16,Paid Social,Paid,300024588642,1
22,300025029469,2022-04-17,4,12,bfb7a9d770e4a6683b00256befc9a575,37039.55,16,Paid Search,Paid,300025029469,1


### Now we have the dataset cleaned and ready to be used for Cohort creations

### Part 5:Create a column called Index Month and Year for each Customer ID where

1. If transaction row has flag set to 1 then the year and month of the corresponding transaction will be the index month/year


2. If flag is set to 0 then set the Index month/year values by looking at the row where the customer id has the index/month year flag already set(i.e the first purchase made by the customer)

In [12]:
#First we will rename the rows which designate 1st purchase as Index_month,year,Acquired channel and Channel type
df_index=df_jan22_may23[df_jan22_may23['Flag']==1]

df_index.rename(columns={'Month':'Index_month','Year':'Index_year','First Touch Channel':'Acquired Channel',
                        'Channel_Type':'Acquired Channel Type'},inplace=True)
df_index=df_index[['CustomerID','Index_month','Index_year','Acquired Channel','Acquired Channel Type']]
df_index


#Next we will merge this dataframe with the overalldataframe so that we have index month,year for each line of transaction
# We will call this dataframe cohort

df_cohort=df_index.merge(df_jan22_may23,on=['CustomerID'])
df_cohort.sample(5)

C:\Users\24057\AppData\Local\Temp\ipykernel_20628\3045541801.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_index.rename(columns={'Month':'Index_month','Year':'Index_year','First Touch Channel':'Acquired Channel',


,CustomerID,Index_month,Index_year,Acquired Channel,Acquired Channel Type,OrderNo,OrderDate,Month,Year,Revenue,Seq,First Touch Channel,Channel_Type,FirstOrderNo,Flag
13576,d81866a734674054bffeadc709f51842,5,12,TCP,Paid,TC300027915381,2022-06-12,6,12,4499.1,18,Internal,Organic,TC300027010463,0
103861,0ace07508ea8999a6f4f140ac9758828,1,12,Paid Search,Paid,300021825259,2022-01-16,1,12,899.0,13,Paid Search,Paid,300021825259,1
43747,66bf01c5fb5c666ef78f065a7030e3b0,4,12,Affiliate,Paid,300025531703,2022-04-29,4,12,6299.1,16,Affiliate,Paid,300025531703,1
56169,11cd9e7130a1a08cd885cfb3c6d37531,3,12,Paid Search,Paid,300028597768,2022-06-23,6,12,527.0,18,Direct,Organic,300023292854,0
69326,9399b072f49878b432010ea8878db50b,4,12,Paid Search,Paid,300025365507,2022-04-25,4,12,7840.0,16,Paid Search,Paid,300025365507,1


In [13]:
#We will create a column called Index_Seq which will be the first month+year and Current_Seq for Current Month+Year
df_cohort['Index_Seq']=df_cohort['Index_month']+ df_cohort['Index_year']
df_cohort['Current_Seq']=df_cohort['Month']+ df_cohort['Year']

#Create a column - Cohort which will contain the cohort number by subtracting the First month from current month
#Since we are doing this only for first 5 months of 2022 we are interested only in the first 5 index months
df_cohort=df_cohort[df_cohort['Index_Seq']<=17]

#We will create a column called Cohort which will be the difference in the Current month and the index month
df_cohort['Cohort']=df_cohort['Current_Seq']-df_cohort['Index_Seq']
df_cohort=df_cohort[df_cohort['Cohort']<=12]
df_cohort['Index_Seq']=df_cohort['Index_Seq']-12

#We will reatain ony columns which are relevant to our cohort calculation
df_cohort=df_cohort[['CustomerID','Index_Seq','Cohort','Revenue','OrderNo','Acquired Channel','Acquired Channel Type']]
df_cohort.head()

,CustomerID,Index_Seq,Cohort,Revenue,OrderNo,Acquired Channel,Acquired Channel Type
0,669a59c305f29fdd332fcf85d2b41d66,5,0,5509.80,300026821121,Paid Search,Paid
1,133afa43f4aca20d1541b47bbbceea59,5,0,44990.00,TC300025956228,TCP,Paid
2,ba789948cd094725bde4064809c8d25c,5,0,54990.00,TC300026763825,TCP,Paid
3,a0461e874d88d8ff4df469ba5bb1e108,4,0,3134.05,300024588642,Paid Social,Paid
4,bfb7a9d770e4a6683b00256befc9a575,4,0,37039.55,300025029469,Paid Search,Paid


Now, we have the following data about each customer 
1. What was the index month that this customer made the purchase
2. What was the Revenue generated by this transaction
3. What was the Channel which drove this transaction
4. In what cohorts do all transactions beyond the 1st transaction map out



### Part 6: Calculate CLV of each cohort

1. Now we can find out the revenue generated by each customer in an index in each cohort

2. Also we can find out the total customers in an index

If we divide 1 by 2 we can get the net CLV generated by each customer in each of the 5 index


In [14]:
#Enter a channel we will do this for
channel=input('Please enter channel (Affiliate/Paid Search etc) ')
df_cohort=df_cohort[df_cohort['Acquired Channel']==channel]

#Caluclate sum of Revenue by customers in Each Index and each cohort
df_cohort_1=df_cohort.groupby(by=['Index_Seq','Cohort']).sum().reset_index()

#Caluclate count of customers in Each Index and each cohort
df_cohort_2=df_cohort.groupby(by=['Index_Seq','Cohort'])['OrderNo'].count().reset_index()
df_cohort_3=df_cohort.groupby(by=['Index_Seq','Cohort'])['CustomerID'].nunique().reset_index()
df_cohort_2.rename(columns={'OrderNo':'Order_count'},inplace=True)
df_cohort_3.rename(columns={'CustomerID':'Customer_count'},inplace=True)

#create a Revenue per customer column which will give average value of a customer in each cohort
x=df_cohort_2.merge(df_cohort_3,on=['Index_Seq','Cohort'])
df_cohort=df_cohort_1.merge(x,on=['Index_Seq','Cohort'])
df_cohort['RevinCr']=round(df_cohort['Revenue']/10000000,2)


#Step 5: Pivot this dataframe where rows=Index_month, columns='Cohort',values='Count'
pd.options.display.float_format = '{:.2f}'.format
df_rev_pivot=df_cohort.pivot_table(index='Index_Seq',columns='Cohort',values='RevinCr')
df_cust_pivot=df_cohort.pivot_table(index='Index_Seq',columns='Cohort',values='Customer_count')

Please enter channel (Affiliate/Paid Search etc) Paid Search


C:\Users\24057\AppData\Local\Temp\ipykernel_20628\365803311.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_cohort_1=df_cohort.groupby(by=['Index_Seq','Cohort']).sum().reset_index()


<IPython.core.display.Javascript object>

In [15]:
df_rev_pivot

Cohort,0,1,2,3,4,5,6,7,8,9,10,11,12
Index_Seq,,,,,,,,,,,,,
1,11.70,0.00,NaN,NaN,NaN,0.11,0.06,0.05,0.10,0.10,0.08,0.03,0.03
2,8.45,0.00,NaN,NaN,0.15,0.08,0.07,0.09,0.09,0.07,0.04,0.07,0.04
3,19.61,0.00,0.01,0.17,0.13,0.08,0.14,0.13,0.09,0.14,0.09,0.04,0.11
4,23.68,0.00,0.21,0.16,0.11,0.14,0.15,0.12,0.17,0.09,0.08,0.18,0.17
5,33.53,0.45,0.23,0.12,0.23,0.34,0.20,0.21,0.18,0.11,0.22,0.31,0.29


In [16]:
df_cust_pivot

Cohort,0,1,2,3,4,5,6,7,8,9,10,11,12
Index_Seq,,,,,,,,,,,,,
1,9684.00,1.00,NaN,NaN,NaN,79.00,54.00,28.00,64.00,63.00,58.00,41.00,37.00
2,8392.00,1.00,NaN,NaN,73.00,46.00,34.00,41.00,61.00,55.00,37.00,48.00,36.00
3,14942.00,1.00,2.00,104.00,88.00,75.00,78.00,80.00,72.00,80.00,90.00,57.00,77.00
4,16364.00,1.00,151.00,85.00,69.00,94.00,101.00,95.00,97.00,80.00,73.00,96.00,105.00
5,22256.00,270.00,158.00,121.00,166.00,168.00,145.00,144.00,129.00,86.00,140.00,142.00,163.00


In [17]:
#CLV of each customer acquired in Jan 2022 via Paid Search

#Net revenue generated by each cohort
cohort_revenue=df_rev_pivot.sum(axis=1)*10000000

#Total customer acquired in each cohort
cohort_count=df_cust_pivot[0]

print('CLV for',channel,':',cohort_revenue/ cohort_count)


CLV for Paid Search : Index_Seq
1   12660.06
2   10903.24
3   13880.34
4   15436.32
5   16364.13
dtype: float64
